In [3]:
pwd

'/home/karish19471/btp/amr'

### Converting the AMR graphs into the required format

In [4]:
import glob
import networkx as nx
import penman
import amrlib
import pandas as pd
import penman
from penman import constant
from amrlib.graph_processing.annotator import add_lemmas
from amrlib.alignments.rbw_aligner import RBWAligner
from penman.models.noop import NoOpModel
import ast
import pickle
import os
import dgl
import json
import numpy as np
from sklearn.model_selection import train_test_split

/home/karish19471/.local/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [102]:
DATASET = "twitter16"

In [103]:
merged_amr = glob.glob(f"{DATASET}_amr/{DATASET}_amr_merge/*.amr.penman")
df = pd.read_csv(f'{DATASET}_amr/{DATASET}.csv')
# df = pd.read_csv(f'{DATASET}_amr/{DATASET}.tsv', sep = '\t')

In [87]:
def var2word(p_graph):
    v2w = {}
    for (source, _, target) in p_graph.instances():
        v2w[source] = target
    return v2w

In [78]:
def get_glove():
    glove = {}
    f = open('/home/karish19471/glove/glove.6B.100d.txt')
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        glove[word] = coefs
    return glove

In [28]:
def to_dict(d):
    return {i: {'feat':j} for i,j in d.items()}

In [65]:
def id2label(df):
    return dict(zip(df['id'], zip(df['labels'], df['text'])))

In [15]:
glove = get_glove()

In [104]:
i2l = id2label(df)

In [105]:
EMBEDDING_DIM = 100
dataset = []
lv = 0
for curr in merged_amr:
    print(lv)
    p_graph = penman.load(curr, model = NoOpModel())[0]
    name = curr[curr.rfind('/')+1:curr.rfind('.amr')]
    v2w = var2word(p_graph)
    nx_graph = nx.MultiDiGraph()
    nx_graph.add_edges_from([(s, t) for s, _, t in p_graph.edges()])#TODO: Add edges from instances as well

    #-----------------------------------extracting subgraphs----------------------------------------------------
    #sorted ordering is a must in order to preserve the node order in case of using from_networkx
    temp= nx.convert_node_labels_to_integers(nx_graph, ordering = 'sorted', label_attribute= 'original')
    original2new  = {temp.nodes[i]['original']:i for i in temp.nodes}
    subgraphs = [[ original2new[j] for j in i] for i in eval(p_graph.metadata['subgraphs'])]
    #-----------------------------------------------------------------------------------------------------------

    MAP = {i:glove.get(v2w[i], [0]*EMBEDDING_DIM) for i in nx_graph.nodes()}
    attr= to_dict(MAP)
    nx.set_node_attributes(nx_graph, attr)
    try:
        dgl_graph = dgl.from_networkx(nx_graph, node_attrs=['feat'])
    except:
        continue
    (source, target) = dgl_graph.edges()
    label, content = i2l[name]
    sample = {'label':label, 'graph': dgl_graph, 'content': content, 'id': name, 'subgraphs':subgraphs}
    dataset.append(sample)
    lv+=1

ignoring epigraph data for duplicate triple: ('c13-14', ':ARG1', 'c13-11')


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16


ignoring epigraph data for duplicate triple: ('c23-3', ':ARG0', 'c23-2')


17
18
19
20
21
22
23
24


ignoring epigraph data for duplicate triple: ('c5-5', ':mod', 'c5-8')
ignoring epigraph data for duplicate triple: ('c26-3', ':name', 'c26-5')
ignoring epigraph data for duplicate triple: ('c41-2', ':name', 'c41-3')


...

290


ignoring epigraph data for duplicate triple: ('c12-5', ':ARG1', 'c12-1')


291
292
293
294
295
296
297
298
299
300
301
302
303
304
305
306
307
308
309
310
311
312
313
314
315
316
317
318
319
320
321
322
323
324
325
326
327


ignoring epigraph data for duplicate triple: ('c47-5', ':mod', 'c47-1')
ignoring epigraph data for duplicate triple: ('c9-20', ':COREF', 'c10-4')
ignoring epigraph data for duplicate triple: ('c14-9', ':mod', 'c14-9')


328
329
330
331
332
333
334
335
336
337
338
339
340
341
342
343
344
345
346
347
348
349
350
351
352
353
354
355
356
357
358
359
360
361


In [106]:
with open(f"{DATASET}_amr/{DATASET}.pkl", "wb") as f:
    pickle.dump(dataset, f)

In [107]:
with open(f"{DATASET}_amr/{DATASET}.pkl", "rb") as f:
    d =pickle.load(f)

In [108]:
train, test = train_test_split(d, stratify=np.array([i['label'] for i in d]), test_size=0.10)

In [109]:
with open(f"{DATASET}_amr/{DATASET}_train.pkl", "wb") as f:
    pickle.dump(train, f)
with open(f"{DATASET}_amr/{DATASET}_test.pkl", "wb") as f:
    pickle.dump(test, f)